# RAG Thai
ใช้ ChatGPT ช่วยสร้างโค้ดบางส่วนยังไม่แน่ใจว่ามันทำงานถูกหรือเปล่า
- Thai: ลองเพิ่มการจัดการภาษาไทย normalize tokenize split chunk
- embeding : ใช้แบบ Opensource เช่น intfloat/multilingual-e5-base
- Database : faiss เป็นแบบ hybrid
- Generate : ใช้ Gemini 

ให้ผลแม่นยำกว่า [simple_rag.ipynb](./simple_rag.ipynb) ในการค้นร้านอาหาร

In [69]:
!uv add numpy google-genai pythainlp sentence-transformers faiss-cpu rank-bm25 ipywidgets

Resolved 162 packages in 7ms
Audited 158 packages in 0.19ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [70]:
import os
import json
from google import genai
# from google.genai.types import EmbedContentConfig
from pythainlp import word_tokenize
from pythainlp.util import normalize
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import re

if not os.environ.get("GEMINI_API_KEY"):
    raise ValueError("Please set the GEMINI_API_KEY environment variable")

client = genai.Client()


## Text utils
ฟังก์ชั่นสำหรับจัดการข้อความ normalize, token, split, chunk

In [71]:
def th_normalize(text: str) -> str:
    # ลบซ้ำวรรณยุกต์/สระยาว, normalize รูปแบบ, ตัดช่องว่างเกิน
    t = normalize(text or "")
    t = re.sub(r"[^\S\r\n]+", " ", t).strip()
    # ลดเสียงลากยาว "มากกกก" -> "มาก"
    t = re.sub(r"(.)\1{2,}", r"\1", t)
    return t

def th_tokenize(text: str):
    # เลือก engine 'newmm' ซึ่งแม่นยำดีสำหรับไทย
    return word_tokenize(text, engine="newmm")

def sent_split_th(text: str):
    # แยกเป็นประโยคแบบง่าย ๆ ตามเครื่องหมายวรรคตอนไทย/อังกฤษ
    # (สำหรับงานจริง อาจใช้ประโยคจาก Markdown heading/ย่อหน้าร่วมด้วย)
    return re.split(r"(?<=[\.\?\!]|[ๆฯ])\s+", text.strip())

# -------- Chunking ตามประโยค + ขนาดโทเคนคร่าว ๆ --------
def chunk_by_sentences(text, max_chars=600, overlap_chars=80):
    sents = [s for s in sent_split_th(th_normalize(text)) if s]
    chunks = []
    buf = ""
    for s in sents:
        if len(buf) + len(s) + 1 <= max_chars:
            buf = (buf + " " + s).strip()
        else:
            if buf:
                chunks.append(buf)
            # overlap
            if overlap_chars > 0 and chunks:
                tail = chunks[-1][-overlap_chars:]
                buf = (tail + " " + s).strip()
            else:
                buf = s
    if buf:
        chunks.append(buf)
    return chunks

def chunk_documents(docs, max_chars=600, overlap_chars=80):
    all_chunks = []
    for doc in docs:
        chunks = chunk_by_sentences(doc["description"], max_chars, overlap_chars)
        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "id": f"{doc['id']}_chunk{i+1}",
                "title": doc.get("name", ""),
                "text": chunk
            })
    return all_chunks


ทดสอบ text utils

In [72]:
# test_thai_word = "กรุงเทพฯ 5 มี.ค. - ธนาคารแห่งประเทศไทย (ธปท.) \nเตือนประชาชนระวังมิจฉาชีพแอบอ้างเป็นเจ้าหน้าที่ธนาคารหรือหน่วยงานรัฐ ส่ง SMS หรือ LINE ที่มีลิงก์ปลอม เพื่อหลอกให้โอนเงินหรือติดตั้งแอปพลิเคชันอันตราย พร้อมแนะวิธีป้องกันตัวเองจากมิจฉาชีพ"
test_thai_word = "สวัสดีมากๆ!เลย    จ้าาาา"
print("normalize:",th_normalize(test_thai_word) )
print("tokenize:",th_tokenize(test_thai_word))
print("split:",sent_split_th(test_thai_word))
print("norm+token:",th_tokenize(th_normalize(test_thai_word)))
print("chunk_by_sentences:",chunk_by_sentences(test_thai_word))

normalize: สวัสดีมากๆ!เลย จ้า
tokenize: ['สวัสดี', 'มาก', 'ๆ', '!', 'เลย', '    ', 'จ้า', 'าาา']
split: ['สวัสดีมากๆ!เลย    จ้าาาา']
norm+token: ['สวัสดี', 'มาก', 'ๆ', '!', 'เลย', ' ', 'จ้า']
chunk_by_sentences: ['สวัสดีมากๆ!เลย จ้า']


## Embedding
ตัวอย่างนี้ใช้ intfloat/multilingual-e5-base หรือ BAAI/beg-m3 รองรับหลายภาษารวมทั้งภาษาไทย ตั้งไว้แบบ CPU เพื่อจะได้ไม่มี

In [73]:
#embed_model = SentenceTransformer("intfloat/multilingual-e5-base",device='cpu')
embed_model = SentenceTransformer("BAAI/bge-m3",device='cpu')
def embed(texts):
    return embed_model.encode([f"{t}" for t in texts], normalize_embeddings=True,device='cpu')
    

In [74]:
def create_chunk_records(docs):
    # 3) สร้างชิ้นข้อมูล (chunks)
    chunk_records = []
    for d in docs:
        chunks = chunk_by_sentences(d["description"], max_chars=500, overlap_chars=50)
        for i, c in enumerate(chunks):
            chunk_records.append({
                "doc_id": d["id"],
                "title": d["name"],
                "chunk_id": f'{d["id"]}-{i}',
                "text": c,
            })
    return chunk_records
def create_embedding(chunk_records):
    emb_matrix = embed([f"{r["title"]}:\n{r["text"]}"  for r in chunk_records]).astype("float32")
    # ดัชนีเวคเตอร์ FAISS
    d = emb_matrix.shape[1]
    index = faiss.IndexFlatIP(d)  # ใช้ cosine-sim กับเวคเตอร์ normalized = dot product
    index.add(emb_matrix)
    return index

## Hybrid Search


In [ ]:
def hybrid_search(query,index,chunk_records, top_k=6, alpha=0.5):
    """
    alpha: ถ้า 1.0 เน้น vector ล้วน, ถ้า 0.0 เน้น BM25 ล้วน
    """
    q_norm = th_normalize(query)
    q_tokens = th_tokenize(q_norm)

    #BM25
    bm25_corpus_tokens = [th_tokenize(r["text"]) for r in chunk_records]
    bm25 = BM25Okapi(bm25_corpus_tokens)
    bm25_scores = bm25.get_scores(q_tokens)
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:top_k*3]  # ขยายเผื่อรวมกับเวคเตอร์

    # 7.2 Vector
    q_emb = embed_model.encode([f"query: {q_norm}"], normalize_embeddings=True).astype("float32")
    sim, idx = index.search(q_emb, top_k*3)  # cosine-sim (dot)
    vec_scores = np.zeros(len(chunk_records))
    vec_scores[idx[0]] = sim[0]

    # 7.3 รวมคะแนน (normalize แบบ min-max)
    def norm(x):
        x = np.array(x, dtype=np.float32)
        if x.max() - x.min() < 1e-8:
            return np.zeros_like(x)
        return (x - x.min()) / (x.max() - x.min())

    bm25_mask = np.zeros(len(chunk_records)); bm25_mask[bm25_top_idx] = 1
    union_idx = np.where((bm25_mask + (vec_scores>0))>0)[0]

    score_b = norm(bm25_scores[union_idx])
    score_v = norm(vec_scores[union_idx])
    score = alpha*score_v + (1-alpha)*score_b

    order = union_idx[np.argsort(score)[::-1][:top_k]]
    results = []
    for i in order:
        r = chunk_records[i]
        results.append({
            "chunk_id": r["chunk_id"],
            "doc_id": r["doc_id"],
            "title": r["title"],
            "text": r["text"],
            "score": float(score[np.where(union_idx==i)][0]),
        })
    return results

## Generate

In [ ]:
# สร้างคำตอบโดยยึดตามหลัก "ใช้เฉพาะจากบริบท" ใช้ contexts เหมือนจะดีกว่า context_text
def generate_response(query, contexts,max_ctx_chars=1200):
    """Generate a response using Gemini based on the query and retrieved context"""
    context_text = ""
    for c in contexts:
        if len(context_text) + len(c["text"]) + 2 <= max_ctx_chars:
            context_text += f"- {c['text']}\n"

    prompt = f"""
You are a helpful assistant .
If the query asks for something not in the provided information, politely indicate 
that you don't have that specific information but suggest the closest alternatives.
Please provide a helpful response that directly answers the user's query based on information below.

USER QUERY: {query}

INFORMATION:
{contexts}

"""
    
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=prompt,
    )
    answer = response.text
    return prompt, answer


Reload Data

In [120]:

docs=[]
with open('./data1.json') as f:
    docs = json.load(f)
chunk_records = create_chunk_records(docs)
index = create_embedding(chunk_records)

RAGS ลองปรับ user_query และ alpha ว่าผลแตกต่างกันหรือไม่

In [121]:
# user_query = "ถ้าโอนเงินผิดบัญชีควรทำอย่างไร และควรระวังมิจฉาชีพแบบไหน?"
user_query = "ร้านอาหารอะไรมีเมนูอาหารไทย"
# user_query = "ถ้าได้ SMS น่าสงสัยควรทำอย่างไร"
results = hybrid_search(user_query,index,chunk_records, top_k=5, alpha=0.6)
prompt, answer = generate_response(user_query, results)

data2.json : doc9 ถูก Chunking ออกมาได้ไม่ดีซักเท่าไหร่

In [123]:
chunk_records

[{'doc_id': '001',
  'title': 'La Dotta',
  'chunk_id': '001-0',
  'text': 'Authentic Italian restaurant specializing in handmade pasta dishes using imported ingredients from Italy. Located in Thonglor district of Bangkok.'},
 {'doc_id': '002',
  'title': 'Peppina',
  'chunk_id': '002-0',
  'text': 'Neapolitan pizza restaurant with wood-fired ovens imported from Italy. Features traditional Italian dishes and a wide selection of Italian wines.'},
 {'doc_id': '003',
  'title': "L'Oliva",
  'chunk_id': '003-0',
  'text': 'High-end Italian dining with focus on Northern Italian cuisine. Offers homemade pasta, risotto, and seafood specialties in the heart of Sukhumvit.'},
 {'doc_id': '004',
  'title': 'Appia',
  'chunk_id': '004-0',
  'text': 'Roman-inspired trattoria serving hearty Italian comfort food including porchetta and homemade pasta in a rustic setting in Sukhumvit Soi 31.'},
 {'doc_id': '005',
  'title': 'Pizza Massilia',
  'chunk_id': '005-0',
  'text': 'Upscale Italian restaurant

In [130]:
print("Top Hits:")
for i, r in enumerate(results):
    print(f"{i+1} {r['title']} ({r['score']:.3f})")
    print(f"{r['text'][:120]}...")

Top Hits:
1 Isaan Der (0.600)
Northeastern Thai cuisine featuring grilled meats, sticky rice, and spicy salads served in a casual atmosphere near Asok...
2 Pad Thai Ekkamai (0.600)
Local favorite serving authentic pad thai and other traditional Thai noodle dishes in the trendy Ekkamai neighborhood....
3 Appia (0.600)
Roman-inspired trattoria serving hearty Italian comfort food including porchetta and homemade pasta in a rustic setting ...
4 Som Tam Nua (0.600)
Popular Thai restaurant specializing in Northeastern Thai cuisine, especially spicy papaya salad and grilled chicken. Lo...
5 Gianni's (0.600)
Classic Italian restaurant in Sukhumvit offering traditional dishes from various regions of Italy, with an extensive win...


In [125]:
print("Prompt:\n", prompt)

Prompt:
 
You are a helpful assistant .
If the query asks for something not in the provided information, politely indicate 
that you don't have that specific information but suggest the closest alternatives.
Please provide a helpful response that directly answers the user's query based on information below.

USER QUERY: ร้านอาหารอะไรมีเมนูอาหารไทย

INFORMATION:
[{'chunk_id': '009-0', 'doc_id': '009', 'title': 'Isaan Der', 'text': 'Northeastern Thai cuisine featuring grilled meats, sticky rice, and spicy salads served in a casual atmosphere near Asok.', 'score': 0.6000000238418579}, {'chunk_id': '008-0', 'doc_id': '008', 'title': 'Pad Thai Ekkamai', 'text': 'Local favorite serving authentic pad thai and other traditional Thai noodle dishes in the trendy Ekkamai neighborhood.', 'score': 0.6000000238418579}, {'chunk_id': '004-0', 'doc_id': '004', 'title': 'Appia', 'text': 'Roman-inspired trattoria serving hearty Italian comfort food including porchetta and homemade pasta in a rustic setti

In [126]:
print("ถาม:",user_query,"\n ตอบ:\n",answer)

ถาม: ร้านอาหารอะไรมีเมนูอาหารไทย 
 ตอบ:
 ร้านอาหารที่มีเมนูอาหารไทย ได้แก่:

*   **Isaan Der**: มีอาหารอีสาน (Northeastern Thai cuisine) เช่น เนื้อย่าง ข้าวเหนียว และส้มตำรสจัด
*   **Pad Thai Ekkamai**: มีผัดไทยต้นตำรับและอาหารประเภทเส้นไทยอื่นๆ
*   **Som Tam Nua**: เป็นร้านอาหารไทยที่เชี่ยวชาญอาหารอีสาน โดยเฉพาะส้มตำรสจัดและไก่ย่าง
